In [6]:
import torch
print(torch.cuda.is_available())

False


In [7]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams


max_model_len, tp_size = 4096, 1
model_name = "neuralmagic/granite-3.1-2b-base-quantized.w4a16"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
llm = LLM(model=model_name, tensor_parallel_size=tp_size, max_model_len=max_model_len, trust_remote_code=True, device='cpu')
sampling_params = SamplingParams(temperature=0.3, max_tokens=256, stop_token_ids=[tokenizer.eos_token_id])

Device: cpu
INFO 03-10 23:36:13 config.py:549] This model supports multiple tasks: {'score', 'classify', 'embed', 'reward', 'generate'}. Defaulting to 'generate'.


NotImplementedError: 

In [ ]:
messages_list = [
    [{"role": "user", "content": "Who are you? Please respond in pirate speak!"}],
]

prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True) for messages in messages_list]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
generated_text = [output.outputs[0].text for output in outputs]
print(generated_text)

In [3]:
from huggingface_hub import snapshot_download
model_path = "ibm-granite/granite-3.1-1b-a400m-base"
# model_path = "DevQuasar/ibm-granite.granite-3.1-1b-a400m-instruct-GGUF"
snapshot_download(repo_id=model_path, repo_type="model")

Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 2513.97it/s]


'/home/dev/.cache/huggingface/hub/models--ibm-granite--granite-3.1-1b-a400m-base/snapshots/408b6e90baab8cf24f4aa9f8e19703ffa0a53b29'

In [4]:
from pydantic import BaseModel, Field

class SecurityEvent(BaseModel):
    # The reasoning for why this event is relevant.
    reasoning: str

    # The type of event.
    event_type: str

    # Whether this event requires human review.
    requires_human_review: bool

    # The confidence score for this event. I'm not sure if this
    # is meaningful for language models, but it's here if we want it.
    confidence_score: float = Field(
        ge=0.0, 
        le=1.0,
        description="Confidence score between 0 and 1"
    )

    # Recommended actions for this event.
    recommended_actions: list[str]

class HardwareFailureEvent(BaseModel):
    # The reasoning for why this event is relevant.
    reasoning: str

    # The type of event.
    event_type: str

    # Whether this event requires human review.
    requires_human_review: bool

    # The confidence score for this event. I'm not sure if this
    # is meaningful for language models, but it's here if we want it.
    confidence_score: float = Field(
        ge=0.0, 
        le=1.0,
        description="Confidence score between 0 and 1"
    )

    # Recommended actions for this event.
    recommended_actions: list[str]

class LogAnalysis(BaseModel):
    # A summary of the analysis.
    summary: str

    # Observations about the logs.
    observations: list[str]

    # Planning for the analysis.
    planning: list[str]

    # Security events found in the logs.
    security_events: list[SecurityEvent]

    # harware failure events found in the logs.
    Hardware_failure_events: list[HardwareFailureEvent]

    # # Traffic patterns found in the logs.
    # traffic_patterns: list[WebTrafficPattern]

    # # The highest severity event found.
    # highest_severity: Optional[SeverityLevel]
    requires_immediate_attention: bool

In [6]:
from pydantic import BaseModel, Field
import torch
x = torch.rand(5, 3)
print(x)

from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
# model_path = "ibm-granite/granite-3.1-3b-a800m-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
# drop device_map if running on CPU
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)
model.eval()


# change input text as desired
chat = "Please list one IBM Research laboratory located in the United States. You should only output its name and location."
# chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# tokenize the text

prompt_template_path = "prompt_template.txt"

with open(prompt_template_path, "r") as file:
    prompt_template = file.read()


import pandas as pd

df = pd.read_csv('test2.nogit.csv')
# Create a list to store the text lines
text_lines = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    if(index > 200):
        break
    action = "threw" if row['classification'] != "info" else "output"
    text_line = f"LOGID-{index} {row['timestamp']} application: {row['app_name']} in namespace: {row['namespace_name']} {action} {row['classification']}: {row['message']}"
    text_lines.append(text_line)

# Print the first 5 lines of text_lines
for line in text_lines[:5]:
    print(line)

chat = prompt_template.format(
                log_type="application",
                logs=text_lines,
                model_schema=LogAnalysis.model_json_schema(),
                stress_prompt="""You are a computer security intern that's really stressed out. 
                
                Use "um" and "ah" a lot.""",
            )

input_tokens = tokenizer(chat, return_tensors="pt").to(device)
# generate output tokens
output = model.generate(**input_tokens, 
                        max_new_tokens=100)
# decode output tokens into text
output = tokenizer.batch_decode(output)
# print output
print(output)

tensor([[0.9408, 0.0545, 0.9876],
        [0.7080, 0.7548, 0.7764],
        [0.8012, 0.5342, 0.6806],
        [0.9039, 0.4839, 0.2694],
        [0.4924, 0.8897, 0.0599]])


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.50it/s]


LOGID-0 2025-03-09T04:52:20.600177813Z application: legacycrm in namespace: myapp threw warning: The 'ExportToCSV' feature is outdated. Please migrate to 'ExportToXLSX' by the end of Q3.
LOGID-1 2025-03-09T04:53:17.407427133Z application: billing in namespace: myapp output info: application is up and running within acceptable parameters
LOGID-2 2025-03-09T04:59:10.669781089Z application: legacycrm in namespace: myapp threw warning: Support for legacy authentication methods will be discontinued after 2025-06-01.
LOGID-3 2025-03-09T04:59:20.671493491Z application: legacycrm in namespace: myapp threw warning: Support for legacy authentication methods will be discontinued after 2025-06-01.
LOGID-4 2025-03-09T04:59:40.674811633Z application: legacycrm in namespace: myapp threw warning: API endpoint 'getCustomerDetails' is deprecated and will be removed in version 3.2. Use 'fetchCustomerInfo' instead.
['You are an expert security analyst reviewing application web server logs.\n\nYou are a co